클러스터 별 태그를 로드한 후 가장 유사한 클러스터의 곡들과 비교 ?
- 유사도를 어떻게 판단할 것인가?
- 유사한 클러스터를 몇 개 뽑을 것인가?

감정(슬픔/분노/기쁨/중립/걱정) + 키워드

In [284]:
import sqlite3
import numpy as np
from gensim.models import FastText

In [345]:
class Music_Recommender:
    def __init__(self, 
                 DB = '../flask_api/db.db',
                 fasttext_PATH = './model/trained_fasttext.model'):
        self.DB = DB
        self.model = FastText.load(fasttext_PATH)
    
    def connect2db(self):
        try:
            conn = sqlite3.connect(self.DB)
            cur = conn.cursor()
        except:
            print("DB Connection Error!")
            return None, None
        return conn, cur 
    
    def load_cluster_tags(self):
        conn, cur = self.connect2db()
        cur.execute("SELECT COUNT(*)/10 FROM CLUSTER")
        n_clusters = cur.fetchall()[0][0]
        
        cluster_tags = []
        try:
            for i in range(n_clusters):
                cur.execute("SELECT tag FROM CLUSTER WHERE label = ?", [i])
                cluster_tags.append([tag for tag, in cur.fetchall()])
        except:
            pass
        finally:
            conn.commit()
            conn.close()
        return cluster_tags
    
    def find_similar_clusters(self, keywords):
        cluster_tags = self.load_cluster_tags()
        similarities = []
        
        for cluster_tag in cluster_tags:
            sim = 0
            for tag in cluster_tag:
                for keyword in keywords:
                    sim += self.model.wv.similarity(keyword, tag)
            similarities.append(sim)
        return np.argsort(similarities)[::-1][:len(similarities)//20]

    def run(self, keywords):
        # return musics in @sim_clusters and select randomly with weight on 'like_cnt'
        sim_clusters = self.find_similar_clusters(keywords)
        conn, cur = self.connect2db()
        target_music = []
        print(sim_clusters)
        for label in sim_clusters:
            query = F"SELECT musicId, tag FROM MUSIC WHERE label = {label}"
            cur.execute(query)
            target_music.extend(cur.fetchall())
        
        res, result = [], []
        for musicId, tags in target_music:
            sim = 0
            tags = tags.split()
            for tag in tags:
                for keyword in keywords:
                    sim += model.wv.similarity(tag, keyword)
            res.append([musicId, sim / (len(tags) * len(keywords))])
        
        for m_id in [musicId for musicId, sim in sorted(res, key=lambda x:-x[1])][:20]:
            cur.execute("SELECT title, artist FROM MUSIC WHERE musicId = ?", [m_id])
            result.append(cur.fetchall()[0])
        
        conn.commit()
        conn.close()
        
        return result


In [351]:
recom = Music_Recommender()
recom.run(['행복', '운동'])

[23 31]


[('너라는 명작', '휘성 (Realslow)'),
 ('앞으로 잘 부탁해 (IZ*ONE ver.)', 'IZ*ONE (아이즈원)'),
 ('Firework (불꽃놀이)', '라붐 (LABOUM)'),
 ('시노시작 (SINOSIJAK)', 'iKON'),
 ('Birthday (Don Diablo Remix)', 'Anne-Marie'),
 ('놈놈놈', '써니힐'),
 ('If It&#39;s Over (feat. Charli XCX)', 'MØ'),
 ('Rumor', '국.슈 (국프의 핫이슈)'),
 ('Gucci Rock N Rolla (feat. Rivers Cuomo & KYLE)', 'Snakehips'),
 ('River', 'Adrees'),
 ('Rough Water', 'Travie McCoy Jason Mraz'),
 ('냠냠쩝쩝 (정화&혜린)', 'EXID'),
 ('아파트 (Feat. 나잠수)', '진보'),
 ('보디가드', 'SHINee (샤이니)'),
 ('Happy (Gru`s Theme From Despicable Me 2)', 'Pharrell Williams'),
 ('Havana (Feat. Young Thug)', 'Camila Cabello'),
 ('Feel Special', 'TWICE (트와이스)'),
 ('내가 제일 잘 나가', '2NE1'),
 ('Funkytown', 'Lipps Inc.'),
 ('다이어트 (Feat. 솔지 (EXID))', '프라이머리')]